### 선언적 API
- ex. 함수형 API, 시퀀셜 API
- 사용할 층과 연결 방식을 미리 정의하고, 모델에 데이터를 주입하여 훈련과 추론을 함
- 모델 저장/복사/공유가 쉽고 모델 구조를 분석하기 쉬움. 또한 에러를 일찍 발견할 수 있음.
- **그러나 동적인 구조(반복문, 조건문 등)를 필요로 하는 모델의 경우에는 사용이 어려움.**

## 명령형 API
- 동적인 구조를 필요로하는 경우를 만족시키는 API
- ex. **서브클래싱 API**

In [8]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras

In [17]:
@keras.saving.register_keras_serializable()                       # 데코레이터: 파이썬 전역 메모리 클래스를 등록하고 모델을 불러올 때 이를 통해 찾음
class WideAndDeepModel(tf.keras.Model):                           # tf.keras.Model 상속받아 사용자 정의 모델 클래스 정의
  def __init__(self, units=30, activation="relu", **kwargs):      # 생성자
    super().__init__(**kwargs)                                    # 부모 클래스 생성자 호출
    self.norm_layer_wide = tf.keras.layers.Normalization()        # Wide 경로의 입력을 정규화
    self.norm_layer_deep = tf.keras.layers.Normalization()        # Deep 경로의 입력을 정규화
    self.hidden1 = tf.keras.layers.Dense(units, activation=activation)  # Deep 경로 은닉층1
    self.hidden2 = tf.keras.layers.Dense(units, activation=activation)  # 은닉층 2
    self.main_output = tf.keras.layers.Dense(1)                   # 메인 출력층
    self.aux_output = tf.keras.layers.Dense(1)                    # 보조 출력층

  def call(self, inputs):                                         # 모델 로직 정의 메서드(순전파)
    input_wide, input_deep = inputs
    norm_wide = self.norm_layer_wide(input_wide)                  # 정규화
    norm_deep = self.norm_layer_deep(input_deep)                  # 정규화
    hidden1 = self.hidden1(norm_deep)                             # 은닉층에 통과시킴
    hidden2 = self.hidden2(hidden1)                               # 은닉층에 통과시킴
    concat = tf.keras.layers.concatenate([norm_wide, hidden2])    # 정규화된 Wide 입력과 Deep 은닉층을 서로 연결, 선형 모델과 심층 신경망의 장점을 결합하는 핵심
    output = self.main_output(concat)                             # 최종 예측 생성
    aux_output = self.aux_output(hidden2)                         # 보조 출력 생성
    return output, aux_output                                     # 반환

model = WideAndDeepModel(30, activation="relu", name="my_cool_model")

**유연성:** 서브클래싱 API를 통해 클래스의 call 메서드 내에서 모델을 동적으로 생성할 수 있다.

**검사 불가:** 모델의 구조가 call 메서드 내에 숨어있기 때문에 케라스가 쉽게 검사할 수 없다.<br>
(+) 복제 불가, summary()로 층 연결 확인 불가능

### 모델 저장과 복원

In [18]:
# 모델을 빌드하기 위한 더미 입력 데이터 생성
# Wide & Deep 모델은 두 개의 입력을 받으므로, 각각에 대한 더미 데이터를 만듭니다.
# 각 입력의 특성 수는 임의로 설정할 수 있습니다. 여기서는 5와 10으로 설정합니다.
# 배치 크기는 1로 설정하여 모델을 빌드합니다.
dummy_input_wide = tf.random.normal(shape=(1, 5)) # 예: Wide 경로의 입력 특성 5개
dummy_input_deep = tf.random.normal(shape=(1, 10)) # 예: Deep 경로의 입력 특성 10개

# Normalization 레이어를 미리 'adapt' 시켜줍니다.
# 이렇게 하면 레이어의 평균과 분산이 계산되어 'out of scope' 오류를 방지합니다.
model.norm_layer_wide.adapt(dummy_input_wide)
model.norm_layer_deep.adapt(dummy_input_deep)

# 이제 모델을 한 번 호출하여 빌드합니다.
# 이렇게 하면 모델의 모든 레이어에 대한 가중치가 생성됩니다.
model((dummy_input_wide, dummy_input_deep))

print("모델이 성공적으로 빌드되었습니다.")

모델이 성공적으로 빌드되었습니다.
Saved artifact at 'subclassing_model_tf'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): Tuple[TensorSpec(shape=(None, 5), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)]
Output Type:
  Tuple[TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)]
Captures:
  139415574534800: TensorSpec(shape=(1, 5), dtype=tf.float32, name=None)
  139415574536912: TensorSpec(shape=(1, 5), dtype=tf.float32, name=None)
  139415574537104: TensorSpec(shape=(1, 10), dtype=tf.float32, name=None)
  139415574537872: TensorSpec(shape=(1, 10), dtype=tf.float32, name=None)
  139415574536720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139415574535184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139415574539792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139415574539600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139415

In [19]:
# 모델 저장하기
model.save("subclassing_model.keras")

### tf 파일로 저장하는 이유
- SavedModel 포맷을 사용하여 모델을 저장하기 위해
- 모델의 아키텍쳐와 로직이 직렬화된 계산 그래프 형태로 포함되어 있음(saved_model.pb) <br>
 **-> 제품 환경 배포 시, 모델 정의(Python) 코드가 없어도 실행 가능**
 - keras_metadata.pb : 케라스에 필요한 추가 정보 포함
 - variables/ : 모든 파라미터값 포함
 - assets : 추가 파일(데이터 샘플, 클래스 이름, 특성 이름 등)
 - 이외 옵티마이저도 상태 보존되어 저장

### 저장하기 전에 모델을 빌드하는 이유<br>
: 입력 데이터의 shape을 알면 가중치 행렬을 만들 수 있음.
<br> -> 가중치 행렬을 알아야 저장(.save)할 모델 크기를 초기화할 수 있음. -> 가중치 행렬을 저장할 수 있음
<br> <br>
### 빌드(build) vs 학습(fit)
학습(fit)
- 데이터를 넣어서`epoch = 10` 손실을 줄이는 과정
- **최적값value**을 찾음

빌드(build)
- 모델에게 입력 데이터의 상태를 알려주는 과정
- **입력 데이터의 모양shape**만 랜덤하게 채움


In [21]:
# 모델 로드하기
model = tf.keras.models.load_model(
    "subclassing_model.keras",
    custom_objects={"WideAndDeepModel": WideAndDeepModel}     # 클래스 매핑 정보 주입(Local)
)

X_new_wide = tf.random.normal((3, 5))   # Wide 입력 (5개)
X_new_deep = tf.random.normal((3, 10))  # Deep 입력 (10개)

# 예측 실행
print("데이터 생성 완료! 예측을 시작합니다.")
y_pred_main, y_pred_aux = model.predict((X_new_wide, X_new_deep))

print("\n--- 예측 결과 ---")
print(y_pred_main)

데이터 생성 완료! 예측을 시작합니다.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step

--- 예측 결과 ---
[[14207351.]
 [11136688.]
 [ 8770068.]]


(+) 추가
- save_weights() : 파라미터 값만 저장
- load_weights() : 파라미터 값만 로드<br>
파라미터 = 연결 가중치, 편향, 전처리 통계치, 옵티마이저 상태 등<br>
가중치만 저장하는 것이 전체 모델을 저장하는 것보다 더 빠르고 디스크 공간을 덜 차지함. -> 훈련 중에 체크포인트를 빠르게 저장하는 데 적합
